In [19]:
import yfinance as yf
import pandas as pd

# Download S&P 500 data for the last 5 years
df_sp500 = yf.download("^GSPC", period="max", interval="5d")


[*********************100%***********************]  1 of 1 completed


In [20]:
import numpy as np
import pandas as pd
import time
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, DBSCAN
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


class DTWClustering():
    """
    A class for clustering using Euclidean and FastDTW distances with multiple methods (Hierarchical, KMeans, DBSCAN).
    
    Parameters:
        df (pd.DataFrame): Input dataframe containing time-series data.
    
    Methods:
        validate_dataframe(): Validates the input dataframe.
        standardization(): Standardizes the inputs.
        preprocessing(): Computes percentage change and prepares dataframes for clustering and performance analysis.
        hierarchical_clustering(): Performs clustering using hierarchical methods (Euclidean or FastDTW distance).
        kmeans_clustering(): Performs clustering using KMeans (Euclidean or FastDTW distance).
        dbscan_clustering(): Performs clustering using DBSCAN (Euclidean or FastDTW distance).
        plot_dendrogram(): Plots the dendrogram for hierarchical clustering results.
        analyze_clusters(): Calculates silhouette scores and outputs analysis results.
        elbow_method(): Calculates the optimal number of K for kmeans clustering by plotting the curve to find the elbow.
    """
    
    def __init__(self, df):
        """
        Initialize the DTWClustering object.

        Args:
            df (pd.DataFrame): Input dataframe with time-series data.
        """
        self.df_actual = df
        self.df_pct_change = None
        self.validate_dataframe()

    def validate_dataframe(self):
        """
        Validate the input dataframe to ensure no missing or infinite values.
        """
        if not isinstance(self.df_actual, pd.DataFrame):
            raise ValueError("Input must be a Pandas DataFrame.")
        if self.df_actual.isnull().any().any():
            raise ValueError("Data contains missing values.")
        if not np.isfinite(self.df_actual.values).all():
            raise ValueError("Data contains infinite values.")
        if not np.issubdtype(self.df_actual.dtypes.values[0], np.number):
            raise ValueError("Data contains non-numeric values.")
            
    def preprocessing(self):
        """
        Transform the input dataframe by computing the percentage change for each value 
        relative to the previous row. This creates two dataframes:
        - self.df_actual: The actual values.
        - self.df_pct_change: The percentage change of the actual values.

        Notes:
            - The first row will contain NaN values after the percentage change.
            - NaN values are backward filled to handle missing data.
        """
        # Compute the percentage change
        pct_change_df = self.df_actual.pct_change()

        # Backward fill NaN values that result from the percentage change
        self.df_pct_change = pct_change_df.bfill()

        # Optional: Reset index to keep it clean (if needed)
        self.df_pct_change.reset_index(drop=True, inplace=True)

    def compute_fastdtw_distances(self, df_to_use):
        """
        Compute pairwise FastDTW distances for the dataset.

        Args:
            df_to_use (pd.DataFrame): The dataframe to use for distance calculation (either pct_change or actual values).

        Returns:
            np.ndarray: A condensed distance matrix for hierarchical clustering.
        """
        # Get the number of tickers (columns in df_to_use)
        n = df_to_use.shape[1]
        distances = []

        # Iterate over pairs of columns (tickers)
        for i in range(n):
            for j in range(i + 1, n):
                # Extract the time series for the two tickers (as 1D arrays)
                x = df_to_use.iloc[:, i].values.squeeze()  # Column i
                y = df_to_use.iloc[:, j].values.squeeze()  # Column j

                # Compute the FastDTW distance with Euclidean distance (dist=2)
                dist, _ = fastdtw(x, y, dist=2)
                distances.append(dist)

        return np.array(distances)

    def hierarchical_clustering(self, k=None, distance_metric="euclidean"):
        """
        Perform hierarchical clustering on the percentage change data using a specified distance metric.

        Args:
            k (int, optional): The desired number of clusters. If not provided, 
                                the user must specify the value of k.
            distance_metric (str, optional): The distance metric to use for clustering. 
                                            Can be one of 'euclidean' or 'fastdtw'. 
                                            Default is 'euclidean'.

        Returns:
            dict: A dictionary containing the results of the hierarchical clustering. 
                The dictionary includes:
                - 'linkage_matrix' (ndarray): The linkage matrix resulting from the hierarchical clustering.
                - 'clusters' (ndarray): The cluster labels for each data point.
                - 'silhouette' (float or str): The silhouette score of the clustering, or 'N/A' if the score cannot be computed (e.g., for non-Euclidean distance metrics).
                - 'time' (float): The time elapsed during the clustering process, in seconds.
        """
        start_time = time.time()
        data_for_clustering = self.df_pct_change.values.T


        # Calculate pairwise distances
        if distance_metric == "euclidean":
            pairwise_distances = pdist(data_for_clustering, metric="euclidean")  # Condensed matrix
            method = "ward"
        elif distance_metric == "fastdtw":
            pairwise_distances = self.compute_fastdtw_distances(self.df_pct_change)
            method = "average"
        else:
            raise ValueError(f"Unsupported distance metric: {distance_metric}")

        # Perform hierarchical clustering
        Z = linkage(pairwise_distances, method=method)

        # Ensure k is provided for clustering
        if k is None:
            raise ValueError("Please provide a value for k (number of clusters).")

        # Assign cluster labels
        clusters = fcluster(Z, k, criterion="maxclust")

        # Validate that at least 2 clusters are formed
        unique_clusters = len(set(clusters))
        if unique_clusters <= 1:
            raise ValueError(f"Only {unique_clusters} clusters were formed. Adjust k or check your data.")

        # Compute silhouette score only for Euclidean distances
        if distance_metric == "euclidean":
            silhouette = silhouette_score(self.df_pct_change.values.T, clusters, metric="euclidean")
        else:
            silhouette = "N/A"

        elapsed_time = time.time() - start_time

        # Return clustering results
        return {
            "linkage_matrix": Z,
            "clusters": clusters,
            "silhouette": silhouette,
            "time": elapsed_time
        }

    def kmeans_clustering(self, k, distance_metric="euclidean"):
        """
        Perform KMeans clustering using specified distance metric.

        Args:
            k (int): Number of clusters.
            distance_metric (str): Distance metric ('euclidean' or 'fastdtw').

        Returns:
            dict: Clustering results, including cluster labels, silhouette score, and elapsed time.
        """
        if k <= 0:
            raise ValueError("k must be a positive integer.")

        start_time = time.time()
        
        if distance_metric == "euclidean":
        # Apply KMeans clustering on the percentage change data
            model = KMeans(n_clusters=k)
            clusters = model.fit_predict(self.df_pct_change.values.T)  # Use pct_change data as input
        else:
            raise ValueError("KMeans does not support pairwise distance matrices directly. Consider using another clustering algorithm.")

        # Compute silhouette score
        if distance_metric == "euclidean":
            silhouette = silhouette_score(self.df_pct_change.values.T, clusters, metric="euclidean")
        else:
            silhouette = "N/A"  # Cannot compute silhouette score for non-Euclidean distances
        elapsed_time = time.time() - start_time
        return {"clusters": clusters, "silhouette": silhouette, "time": elapsed_time}

    def dbscan_clustering(self, eps=0.5, min_samples=5, distance_metric="euclidean"):
        """
        Perform DBSCAN clustering using specified distance metric.

        Args:
            eps (float): Maximum distance between two samples to be considered as in the same neighborhood.
            min_samples (int): Minimum number of samples in a neighborhood to form a cluster.
            distance_metric (str): Distance metric ('euclidean' or 'fastdtw').

        Returns:
            dict: Clustering results, including cluster labels and elapsed time.
        """
        start_time = time.time()

        # Compute pairwise distances
        if distance_metric == "euclidean":
            pairwise_distances = pdist(self.df_pct_change.values.T, metric="euclidean")
        elif distance_metric == "fastdtw":
            pairwise_distances = self.compute_fastdtw_distances(self.df_pct_change)
        else:
            raise ValueError("Unsupported distance metric. Use 'euclidean' or 'fastdtw'.")

        # Convert pairwise_distances into a square matrix
        pairwise_distances_square = squareform(pairwise_distances)

        # Apply DBSCAN clustering with the precomputed distance matrix
        model = DBSCAN(metric="precomputed", eps=eps, min_samples=min_samples)
        clusters = model.fit_predict(pairwise_distances_square)

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # Optionally, compute silhouette score, excluding noise points (-1)
        if len(set(clusters)) > 1:  # Make sure there is more than one cluster (excluding noise)
            valid_clusters = clusters != -1
            if valid_clusters.any():  # Ensure there are valid points to calculate silhouette score
                silhouette = silhouette_score(self.df_pct_change.values.T[valid_clusters, :], clusters[valid_clusters], metric="euclidean")
            else:
                silhouette = "N/A"
        else:
            silhouette = "N/A"

        return {"clusters": clusters, "silhouette": silhouette, "time": elapsed_time}
    def plot_dendrogram(self, Z, title="Dendrogram"):
        """
        Plot a dendrogram from a linkage matrix.

        Args:
            Z (array): Linkage matrix.
            title (str): Title for the plot.
        """
        plt.figure(figsize=(10, 6))
        dendrogram(Z)
        plt.title(title)
        plt.xlabel("Cluster Size")
        plt.ylabel("Distance")
        plt.show()
    def analyze_clusters(self, clustering_results, k, title_prefix=""):
        """
        Analyze clusters, calculate and display cluster-level statistics based on actual values.

        Args:
            clustering_results (dict): Output from clustering method.
            k (int): Number of clusters.
            title_prefix (str): Prefix for titles in outputs.
        """
        clusters = clustering_results["clusters"]
        cluster_labels = np.unique(clusters)
        stats = []

        for label in cluster_labels:
            idx = np.where(clusters == label)
            cluster_data = self.df_actual.iloc[idx]
            total_return = cluster_data.sum().sum()
            average = cluster_data.mean().mean()
            variance = cluster_data.var().mean()
            stats.append({"Cluster": label, "Total Return": total_return, "Average": average, "Variance": variance})

        stats_df = pd.DataFrame(stats)
        print(f"\n{title_prefix} Cluster Analysis")
        print(stats_df)

    def elbow_method(self, max_k=10):
        """
        Plot the curve to identify the 'elbow' that corresponds to the optimal number of clusters (K).
        """
        sse = []  # Sum of squared errors for each K
        k_rng = range(1, max_k + 1)  # Test K values from 1 to max_k

        # Compute SSE for each value of K
        for k in k_rng:
            kmeans = KMeans(n_clusters=k, random_state=42)  # Random state for reproducibility
            kmeans.fit(self.df_pct_change.values)  # Fit KMeans to the pct_change data
            sse.append(kmeans.inertia_)  # Append the inertia (SSE)

        # Plot the SSE curve
        plt.figure(figsize=(8, 6))  # Set figure size
        plt.plot(k_rng, sse, linewidth=2, marker='o', linestyle='--', color='b')
        plt.title("Elbow Method for Optimal K")
        plt.xlabel("Number of Clusters (K)")
        plt.ylabel("Sum of Squared Errors (SSE)")
        plt.grid(True)  # Add gridlines for better visibility
        plt.xticks(k_rng)  # Ensure all K values appear on the x-axis
        plt.show()

In [21]:
df_sp500.fillna(method='ffill', inplace=True)  # Forward-fill for existing gaps
df_sp500.fillna(method='bfill', inplace=True)  # Backward-fill for leading gaps
# Take the first 50 timestamps from df_sp500
df_sp500_first_50 = df_sp500.iloc[:10]  # This selects the first 50 rows
# Initialize the DTWClustering with the sliced DataFrame
dtw_clustering = DTWClustering(df_sp500_first_50)

dtw_clustering.preprocessing()

C:\Users\giuli\AppData\Local\Temp\ipykernel_2084\901571968.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sp500.fillna(method='ffill', inplace=True)  # Forward-fill for existing gaps
C:\Users\giuli\AppData\Local\Temp\ipykernel_2084\901571968.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sp500.fillna(method='bfill', inplace=True)  # Backward-fill for leading gaps


In [22]:
df_sp500_first_50

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
1927-12-30 00:00:00+00:00,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-04 00:00:00+00:00,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-09 00:00:00+00:00,17.500000,17.500000,17.500000,17.500000,17.500000,0
1928-01-19 00:00:00+00:00,17.379999,17.379999,17.379999,17.379999,17.379999,0
1928-01-24 00:00:00+00:00,17.709999,17.709999,17.709999,17.709999,17.709999,0
1928-02-03 00:00:00+00:00,17.400000,17.400000,17.400000,17.400000,17.400000,0
1928-02-08 00:00:00+00:00,17.490000,17.490000,17.490000,17.490000,17.490000,0
1928-02-23 00:00:00+00:00,17.129999,17.129999,17.129999,17.129999,17.129999,0


In [23]:
#elbow method for kmeans clustering
dtw_clustering.elbow_method()

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [18]:
# Hierarchical Clustering with FastDTW Distance
print("\nHierarchical Clustering with FastDTW Distance:")
hierarchical_results_fastdtw = dtw_clustering.hierarchical_clustering(k=2, distance_metric="fastdtw")
print(hierarchical_results_fastdtw)


Hierarchical Clustering with FastDTW Distance:


IndexError: tuple index out of range

In [ ]:
# Hierarchical Clustering with Euclidean Distance
print("\nHierarchical Clustering with Euclidean Distance:")
hierarchical_results_euclidean = dtw_clustering.hierarchical_clustering(k=4, distance_metric="euclidean")
print(hierarchical_results_euclidean)

In [ ]:
# KMeans Clustering with Euclidean Distance
print("\nKMeans Clustering with Euclidean Distance:")
kmeans_results_euclidean = dtw_clustering.kmeans_clustering(k=4, distance_metric="euclidean")
print(kmeans_results_euclidean)

In [ ]:
hierarchical_cluster_fastDTW=hierarchical_results_fastdtw["clusters"] -1

hierarchical_cluster_euclidean=hierarchical_results_euclidean["clusters"] -1

dtw_clustering_transposed_actual_values=dtw_clustering.df_actual.T

kmeans_cluster_euclidean=kmeans_results_euclidean["clusters"]


In [ ]:
dtw_clustering_transposed_actual_values['Hierarchical_Cluster_euclidean'] = hierarchical_cluster_euclidean
dtw_clustering_transposed_actual_values['Hierarchical_Cluster_DTW'] = hierarchical_cluster_fastDTW
dtw_clustering_transposed_actual_values['KMeans_Cluster'] = kmeans_cluster_euclidean

In [ ]:
dtw_clustering_transposed_pct_change=dtw_clustering.df_pct_change.T


In [ ]:
dtw_clustering_transposed_pct_change['Hierarchical_Cluster_euclidean'] = hierarchical_cluster_euclidean
dtw_clustering_transposed_pct_change['Hierarchical_Cluster_DTW'] = hierarchical_cluster_fastDTW
dtw_clustering_transposed_pct_change['KMeans_Cluster'] = kmeans_cluster_euclidean

In [ ]:
# Plot dendrograms
print("\nPlotting Dendrogram for Euclidean Hierarchical Clustering:")
dtw_clustering.plot_dendrogram(hierarchical_results_euclidean["linkage_matrix"], title="Euclidean Distance Dendrogram")

print("\nPlotting Dendrogram for FastDTW Hierarchical Clustering:")
dtw_clustering.plot_dendrogram(hierarchical_results_fastdtw["linkage_matrix"], title="FastDTW Distance Dendrogram")

# Analyze clusters for both hierarchical methods
print("\nCluster Analysis for Euclidean Hierarchical Clustering:")
dtw_clustering.analyze_clusters(hierarchical_results_euclidean, k=3, title_prefix="Euclidean Hierarchical")

print("\nCluster Analysis for FastDTW Hierarchical Clustering:")
dtw_clustering.analyze_clusters(hierarchical_results_fastdtw, k=4, title_prefix="FastDTW Hierarchical")
